In [22]:
from tabular_anonymizer import DataFrameAnonymizer
import pandas as pd

In [23]:
file1 = "adult.csv"
df = pd.read_csv(file1, sep=",", index_col=0)
df.reset_index()
df.index = range(len(df))
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [24]:
# Drop some columns and rows
# Keep only 1000 rows
df = df.loc[0:999,]
print(df.shape)
# Drop some columns
df = df.drop(columns = ["fnlwgt", "workclass", "education-num", "occupation", "race", "native-country"])
df.head()

(1000, 15)


,age,education,marital-status,relationship,sex,capital-gain,capital-loss,hours-per-week,label
0,39,Bachelors,Never-married,Not-in-family,Male,2174,0,40,0
1,50,Bachelors,Married-civ-spouse,Husband,Male,0,0,13,0
2,38,HS-grad,Divorced,Not-in-family,Male,0,0,40,0
3,53,11th,Married-civ-spouse,Husband,Male,0,0,40,0
4,28,Bachelors,Married-civ-spouse,Wife,Female,0,0,40,0


In [25]:
# Setup Sensitive attributes you dont want to alter
sensitive_columns = ['label']
# Avg columns
avg_columns = ['capital-gain', 'capital-loss']

# Run tabular_anonymizer
# Set k
k = 5

# List of quasi-identifiers for checking k-anonymity
quasi_identifiers = ['age',	'education', 'marital-status', 'relationship',
	'sex', 'capital-gain', 'capital-loss', 'hours-per-week']

print("Run tabular_anonymizer. Sensitive columns: ", sensitive_columns, ", k=", k)
p = DataFrameAnonymizer(sensitive_columns, avg_columns=avg_columns, format_to_str=True)

# New anonymized dataframe is formed
df_anonymized = p.anonymize_k_anonymity(df, k=k)
df_anonymized.head(10)

Run tabular_anonymizer. Sensitive columns:  ['label'] , k= 5


,age,education,marital-status,relationship,sex,capital-gain,capital-loss,hours-per-week,label,label_count
0,23 - 33,"Assoc-voc, Bachelors, 10th, Prof-school","Divorced, Separated","Unmarried, Not-in-family, Own-child",Male,0.00,0.000000,40 - 56,0,5
1,37 - 60,"Assoc-voc, 5th-6th, 1st-4th, 7th-8th","Never-married, Separated, Married-spouse-absent","Unmarried, Not-in-family",Male,0.00,230.000000,32 - 48,0,5
2,37 - 60,"Assoc-voc, 5th-6th, 1st-4th, 7th-8th","Never-married, Separated, Married-spouse-absent","Unmarried, Not-in-family",Male,0.00,230.000000,32 - 48,1,1
3,22 - 36,"Some-college, 11th, HS-grad","Separated, Married-spouse-absent","Unmarried, Not-in-family, Other-relative",Male,459.25,0.000000,35 - 55,0,8
4,19 - 32,"Some-college, HS-grad","Divorced, Separated, Married-spouse-absent","Not-in-family, Own-child",Female,0.00,0.000000,20 - 38,0,5
5,27 - 36,"10th, HS-grad, Assoc-acdm","Separated, Married-spouse-absent",Unmarried,Female,0.00,0.000000,25 - 42,0,5
6,41 - 72,"Some-college, 9th, 11th, HS-grad","Never-married, Widowed, Married-spouse-absent","Unmarried, Own-child",Female,66.00,0.000000,20 - 40,0,9
7,42 - 71,"11th, Bachelors, Masters, 9th, Some-college, H...","Widowed, Separated, Married-spouse-absent","Unmarried, Other-relative",Male,0.00,411.500000,2 - 50,0,5
8,42 - 71,"11th, Bachelors, Masters, 9th, Some-college, H...","Widowed, Separated, Married-spouse-absent","Unmarried, Other-relative",Male,0.00,411.500000,2 - 50,1,1
9,39 - 58,"1st-4th, Preschool, 12th, 10th",Married-civ-spouse,Husband,Male,0.00,209.666667,40 - 60,0,7


In [31]:
def is_k_anonymous(df, k, quasi_identifiers, count_column):
    for index, row in df.iterrows():
        # If row count < k, search for other identical rows with different sensitive attribute
        if row[count_column] < k:
            query = ' & '.join([f'`{col}` == {row[col]}' for col in quasi_identifiers])
            print(query)
            rows = df.query(query)
            print(rows)
            if sum(rows[count_column]) < k:
                return False
    return True

# Miksi query ei toimi?


In [30]:
columns_to_check = ['sex']

is_k_anonymous(df_anonymized, 5, columns_to_check, 'label_count')

`sex` == Male


UndefinedVariableError: name 'Male' is not defined

Cells below are for testing only (i.e. trash)

In [ ]:
#df_anonymized['sex']
df_anonymized.loc[0, 'sex'] == ['Male']

In [ ]:
df_anonymized.query("`sex` == ['Male']")

In [ ]:
# Only numeric columns
num_columns = ['capital-gain', 'capital-loss']

is_k_anonymous(df_anonymized, 5, num_columns, 'label_count')

In [ ]:
# Original
#def isKAnonymized(df, k):
#    for index, row in df.iterrows():
#        query = ' & '.join([f'{col} == "{row[col]}"' for col in df.columns])
#        rows = df.query(query)
#        if rows.shape[0] < k:
#            return False
#    return True